In [1]:
# TODO: COSMOS, OFA, SBERT, NLI, CLIP device
import sys
import torch
import pandas as pd
from transformers import pipeline
import json
import os
import numpy as np
from tqdm import tqdm
from sbert import sbert
from online_search import retrieve
tqdm.pandas()
folder_contain_test_json = '/root/thesis/dataset/cosmos_anns_acm/acm_anns'
folder_contain_image = '/root/thesis/dataset'

test_data = list(
    map(json.loads, open(os.path.join(folder_contain_test_json,'test_data.json')).readlines())
)
df = pd.DataFrame(test_data)
df['bert_base_score'] = df['bert_base_score'].astype(float)

2022-06-20 20:21:32.435039: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-06-20 20:21:32.435074: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/root/anaconda3/envs/acmmm/lib/python3.7/site-packages/transformers/models/auto/modeling_auto.py:973: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
Some weights of BertForMaskedLM were not initialized from the model checkpoint at binwang/bert-base-nli-stsb and are newly initialized: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bia

device
cuda
cuda
Loading CLIP Model...


In [ ]:
def print_div():
    term_size = os.get_terminal_size()
    print('=' * term_size.columns)

# COSMOS

In [6]:
sys.path.append('/root/thesis/acmmmcheapfakes/COSMOS')
from COSMOS import evaluate_ooc

evaluate_ooc.DATA_DIR = folder_contain_image
evaluate_ooc.JSON_DIR = folder_contain_test_json

evaluate_ooc.main(None)

  proposal_generator.anchor_generator.cell_anchors.0


In [9]:
sys.modules.pop('utils')
sys.modules.pop('utils.eval_utils')

<module 'utils.eval_utils' from '/root/thesis/acmmmcheapfakes/COSMOS/utils/eval_utils.py'>

In [10]:
os.chdir('/root/thesis/acmmmcheapfakes/OFA')
sys.path.remove('/root/thesis/acmmmcheapfakes/COSMOS')
sys.path.append('/root/thesis/acmmmcheapfakes/OFA')

from OFA.main import run
run(df)

In [15]:
cosmos_iou = pd.read_csv('pred_contexts.txt', header=None)
cosmos_iou.columns = ['iou']
df = pd.concat([df, cosmos_iou['iou']], axis=1)

ofa_result = pd.read_csv('ofa_full.csv')
df = pd.concat([df, ofa_result[['c1_entail','c2_entail']]], axis=1)

In [17]:
os.chdir('/root/thesis/acmmmcheapfakes')

# NLI

In [22]:
classifier = pipeline("text-classification", model = "microsoft/deberta-xlarge-mnli",
device=torch.device('cuda',0))

In [23]:
df['nli_label'] = df.progress_apply(lambda x: classifier(x.caption1+x.caption2)[0], axis=1)
df['nli_label_reverse'] = df.progress_apply(lambda x: classifier(x.caption2+x.caption1)[0], axis=1)

  1%|          | 10/1000 [00:00<01:28, 11.18it/s]/root/anaconda3/envs/acmmm/lib/python3.7/site-packages/transformers/pipelines/base.py:1015: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
100%|██████████| 1000/1000 [01:25<00:00, 11.75it/s]


In [24]:
def nli(x):
    if (x.nli_label['label'] == 'CONTRADICTION' and x.nli_label_reverse['label'] == 'CONTRADICTION'):
        return 'CONTRADICTION'
    if (x.nli_label['label'] == 'ENTAILMENT' or x.nli_label_reverse['label'] == 'ENTAILMENT'):
        if (x.nli_label['label'] != 'CONTRADICTION' and x.nli_label_reverse['label'] != 'CONTRADICTION'):
            return 'ENTAILMENT' 
    return 'NEUTRAL'
df['nli'] = df.progress_apply(lambda x:nli(x), axis=1) 

100%|██████████| 1000/1000 [00:00<00:00, 15291.49it/s]


# FAKE

In [26]:
keywords = "fake, hoax, fabrication, supposedly, falsification, propaganda, deflection, deception, contradicted, defamation, lie, misleading, deceive, fraud, concocted, bluffing, made up, double meaning, alternative facts, trick, half-truth, untruth, falsehoods, inaccurate, disinformation, misconception"
df['cap1_mis']=df.progress_apply(lambda x: sbert([x.caption1_modified,keywords]),axis=1)
df['cap2_mis']=df.progress_apply(lambda x: sbert([x.caption2_modified,keywords]),axis=1)

def get_fake_scores(row):
    new_scores = [0,0,0]
    if row['cap1_mis'] > row['cap2_mis'] and row['cap1_mis']>0.15:
        c_fake = row['caption2'].rstrip('.') + ' is not genuine.'
        new_scores[0] = sbert([c_fake,row['caption1']])

        c_fake = row['caption2'].rstrip('.') + ' is fake.'
        new_scores[1] = sbert([c_fake,row['caption1']])

        c_fake = row['caption2'].rstrip('.') + ' wasn\'t true.'
        new_scores[2] = sbert([c_fake,row['caption1']])
    elif row['cap1_mis'] < row['cap2_mis'] and row['cap2_mis']>0.15: 
        c_fake = row['caption1'].rstrip('.') + ' is not genuine.'
        new_scores[0] = sbert([c_fake,row['caption2']])

        c_fake = row['caption1'].rstrip('.') + ' is fake.'
        new_scores[1] = sbert([c_fake,row['caption2']])

        c_fake = row['caption1'].rstrip('.') + ' wasn\'t true.'
        new_scores[2] = sbert([c_fake,row['caption2']])
    return new_scores
df['false_scores'] = df.progress_apply(lambda x: get_fake_scores(x), axis=1)

100%|██████████| 1000/1000 [00:24<00:00, 41.60it/s]


In [70]:
def predict_everything_online(row):
    if row.nli == 'CONTRADICTION':
        return (True,'NLI')
    if row.nli == 'ENTAILMENT':
        return (False,'NLI')
    false_scores = row['false_scores']
    if np.any(np.array(false_scores) >  row['bert_base_score']+0.05):
        return (True, 'MIS')
    if row['iou']>0.25:
        if row['bert_base_score']<0.5:
            c1 = retrieve(os.path.join(folder_contain_image,row['img_local_path']),row['caption1'])[0]
            c2 = retrieve(os.path.join(folder_contain_image,row['img_local_path']),row['caption2'])[0]
            c1 = c1 > 0.97
            c2 = c2 > 0.97
            
            if c1 and c2:
                return (False, 'ONL')

            if (c1):
                if row['c2_entail']>0.25:
                    return (False, 'MAT')
            if (c2):
                if row['c1_entail']>0.25:
                    return (False, 'MAT')

            return (True, 'BERT')
        return (False, 'BERT')
    return (False,'COSMOS')

In [47]:
def evaluate(df, func):
    df['result'] =  df.progress_apply(lambda x:func(x), axis=1)
    df['predict'] =  df['result'].apply(lambda x:x[0])
    df['method'] =  df['result'].apply(lambda x:x[1])
    confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
    print(confusion_matrix)
    result = (confusion_matrix[0][0]+confusion_matrix[1][1])/len(df)
    print('Accuracy:', result)
    method_acc = df.groupby('method').apply(lambda g: \
        ((g['context_label']==g['predict']).sum() / len(g),len(g) ))
    print(method_acc.head(10))

In [71]:
evaluate(df, predict_everything_online)

100%|██████████| 30/30 [01:05<00:00,  2.19s/it]

Actual      0   1
Predicted        
False      17   0
True        3  10
Accuracy: 0.9
method
BERT      (0.8333333333333334, 12)
COSMOS                    (1.0, 4)
MIS                      (0.75, 4)
NLI                       (1.0, 9)
ONL                       (1.0, 1)
dtype: object



/root/anaconda3/envs/acmmm/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/root/anaconda3/envs/acmmm/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/root/anaconda3/envs/acmmm/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [72]:
df.to_csv('result_df.csv', index=False)
df['predict'].to_csv('predict.csv', index=False)

KeyError: 'predict'